<a href="https://colab.research.google.com/github/kushalshah0/colab_tools/blob/main/cess_qr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install pandas openpyxl qrcode[pil]

import pandas as pd
import qrcode
import os
import shutil
from google.colab import files

# Step 1: Upload the Excel file
uploaded = files.upload()

# Assuming the uploaded file is named 'bce.xlsx'
file_path = 'bce.xlsx'  # Change this if your file has a different name

# Step 2: Read the Excel file
df = pd.read_excel(file_path)

# Convert the 'DOB (BS)' column to datetime, if it's not already
df['DOB (BS)'] = pd.to_datetime(df['DOB (BS)'], errors='coerce')

# Step 3: Create a directory to save QR codes and text files
output_dir = 'qr_codes_and_texts'
os.makedirs(output_dir, exist_ok=True)

# Step 4: Initialize a variable to accumulate all entries
all_entries_text = ""

# Step 5: Generate text and QR codes for each entry
for index, row in df.iterrows():
    # Check for missing blood group and replace with '-'
    blood_group = row['Blood Group'] if pd.notna(row['Blood Group']) and row['Blood Group'] != '' else '-'

    # Format DOB to exclude time and convert to string
    dob = row['DOB (BS)'].strftime('%Y-%m-%d') if pd.notna(row['DOB (BS)']) else '-'

    # Format the full name to have the first character of each part capitalized
    full_name = row['Full Name'].title() if pd.notna(row['Full Name']) else '-'

    # Construct the text string
    text = (f"Name: {full_name} "
            f"Roll No: {row['Roll No']} "
            f"Address: {row['Address']} "
            f"Contact No.: {row['Phone No']} "
            f"DOB: {dob} "
            f"Blood Group: {blood_group}\n")

    # Add the text to the accumulated entries with a gap
    all_entries_text += text + "\n"  # Add an extra newline for a gap

    # Generate QR code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(text)
    qr.make(fit=True)

    # Create an image from the QR Code instance
    img = qr.make_image(fill_color="black", back_color="white")

    # Save the image with the roll number in lowercase
    roll_no = row['Roll No'].lower()  # Convert roll number to lowercase
    img_file_path = os.path.join(output_dir, f'{roll_no}.png')
    img.save(img_file_path)

    print(f'Entry {index + 1} saved: {img_file_path}')

# Step 6: Save all entries to a single text file
all_entries_file_path = os.path.join(output_dir, 'all_entries.txt')
with open(all_entries_file_path, 'w') as all_entries_file:
    all_entries_file.write(all_entries_text)

print(f'All entries saved to: {all_entries_file_path}')

# Step 7: Zip the output directory for easy download
shutil.make_archive(output_dir, 'zip', output_dir)

# Step 8: Provide a link to download the zip file
files.download(f'{output_dir}.zip')